In [ ]:
import os
import sys
import time
os.environ['TF_ENABLE_ONEDNN_OPTS'] = str(0)

import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
from tensorflow import keras  # tf.keras
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
print("python", sys.version)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

In [ ]:
assert sys.version_info >= (3, 5) # Python ≥3.5 required
assert tf.__version__ >= "2.0"    # TensorFlow ≥2.0 required

In [ ]:
from test_utility import *

# Start Test

In [ ]:
n_times = 1000; n_markets = 100; n_fields = 10
Candles = [ [ [ time * n_markets * n_fields + market * n_fields + field for field in range(n_fields) ] for market in range(n_markets) ] for time in range(n_times)]
Candles = np.array(Candles, dtype=np.float32)
Times = np.array( range(Candles.shape[0]), dtype=Candles.dtype ) + 333
Times = np.expand_dims(Times, axis=1)
size_time = 1
print(Candles.shape, Times.shape)   # time, market, field
print(Candles[:2, :, :])

In [ ]:
Nx = 10
Ny = 2
Ns = 10

sample_anchors = np.array(range(0, Candles.shape[0] - Nx - Ny + 1, Ns), dtype=np.int32)
sample_anchores_t = sample_anchors[: sample_anchors.shape[0] * 3 // 4]
sample_anchores_v = sample_anchors[sample_anchors.shape[0] * 3 // 4 :]
print(Candles.shape, sample_anchors.shape, sample_anchores_t.shape, sample_anchores_v.shape)

x_indices = ( (0, 1, 2), (0,1,2) )    # (market, field)
y_indices = ( (2, 3, 4), (1,2,3) )    # (market, field). Currently must be the same size as x_indices.
print(Candles[0:2][:, x_indices[0]][:, :, x_indices[1]])
print(Candles[2:4][:, y_indices[0]][:, :, y_indices[1]])

size_x = get_timepoint_size(x_indices)
size_y = get_timepoint_size(y_indices)

print(size_x, size_y, size_time)

In [ ]:
# Dummy variables
CandleMarks = Candles[:, :, 0]
all_market_names = [ 'market_' + str(i) for i in range(Candles.shape[1])]
all_field_names = [ 'filed_' + str(i) for i in range(Candles.shape[2])]
min_true_candle_percent_x = 50
chosen_fields_names_x = all_field_names[2:7]
min_true_candle_percent_y = 50
chosen_fields_names_y = all_field_names[0:5]
target_market_names = all_market_names[2:15] #[0:1] # [1:2]
tarket_market_top_percent = 15

assert min_true_candle_percent_x == min_true_candle_percent_y
assert len(chosen_fields_names_x) == len(chosen_fields_names_y)

Candles, CandleMarks, all_market_names, x_indices, y_indices, \
chosen_market_names_x, chosen_field_names_x, chosen_market_names_y, chosen_field_names_y, \
chosen_market_names, chosen_field_names, \
target_markets_names, target_markets = \
get_formed_data( Candles, CandleMarks, all_market_names, all_field_names, 
        min_true_candle_percent_x, chosen_fields_names_x, min_true_candle_percent_y, chosen_fields_names_y,
        target_market_names, tarket_market_top_percent
)

print(Candles.shape)
print(CandleMarks.shape)
print(len(all_market_names))
print(x_indices)
print(y_indices)
print(chosen_market_names_x)
print(chosen_field_names_x)
print(chosen_market_names_y)
print(chosen_field_names_y)
print(chosen_market_names)
print(chosen_field_names)
print(target_markets_names)
print(target_markets)
print(len(chosen_market_names_x), len(chosen_market_names_y), len(target_markets_names))

In [ ]:
Time_into_X = True
Time_into_Y = False
BatchSize = 2
shuffle_batch = 10

ds_train, ds_valid, dx, dy = \
get_datasets_2(
    Candles, Time_into_X, Time_into_Y, Times, 
    sample_anchores_t, sample_anchores_v,
    Nx, x_indices, Ny, y_indices, size_time, target_markets,
    BatchSize, shuffle_batch, shuffle=True
)

print(len(ds_train), len(ds_valid))

In [ ]:
non_target_markets_relative = [ y_indices[0].index(i) for i in y_indices[0] if i not in target_markets ]
print(non_target_markets_relative)
nFeaturesPerMarket = len(y_indices[1])
print(nFeaturesPerMarket)
all_ntmr = []
for ntmr in non_target_markets_relative:
    all_ntmr = all_ntmr + list(range(ntmr* nFeaturesPerMarket, (ntmr+1) * nFeaturesPerMarket, 1))
non_target_features_relative = tuple(all_ntmr)
print(non_target_features_relative)

def anchor_to_sample(anchor): # This function is the bottle-neck of training speed.
    x = np.reshape(Candles[anchor: anchor + Nx][:, x_indices[0]][:, :, x_indices[1]], (Nx, -1))
    if Time_into_X is True:
        assert Times is not None
        x = np.concatenate((x, np.reshape(Times[anchor: anchor + Nx], (Nx, -1))), axis=1) # concat(x, x_time)

    y = np.reshape(Candles[anchor + Nx: anchor + Nx + Ny][:, y_indices[0]][:, :, y_indices[1]], (Ny, -1))

    if Time_into_X is True:
        assert Times is not None
        y_time = np.reshape(Times[anchor + Nx: anchor + Nx + Ny], (Ny, -1))
        if Time_into_Y is not True: y_time[:] = 0.0
        y = np.concatenate((y, y_time), axis=1)

    x = np.pad(x, [[1,1], [0,0]], constant_values=0)   # (1 pre-pad: Start, 1 post-pad: End) on axis 0. (0 pre-pad, 0 post-pad) on axis 1.
    y = np.pad(y, [[1,1], [0,0]], constant_values=0)

    if x.shape[-1] % 2 != 0:
        x = np.pad(x, [[0,0], [0,1]], constant_values=0) # (0 pre-pad: Start, 0 post-pad: End) on axis 0. (0 pre-pad, 1 post-pad) on axis 1.
        y = np.pad(y, [[0,0], [0,1]], constant_values=0)

    y[:, non_target_features_relative] = 0.0 # Start/Stop = NoInfo = 0.0

    return x, y[:-1], y[1:]

In [ ]:
anchor = 10

x, y, y_shift = anchor_to_sample(anchor)
print(x.shape)
print(y.shape)
print(y_shift.shape)

In [ ]:
print(Candles[:4, :, :]) # market order may be changed by argmax.

In [ ]:
# Complete check of x

for anchor in np.concatenate((sample_anchores_t, sample_anchores_v)):

    x, y, y_shift = anchor_to_sample(anchor)

    assert x.shape[0] == Nx + 2
    size_x = len(chosen_market_names_x)  * len(chosen_field_names_x)
    odd_padding = (1 if size_x // 2 != 0 else 0)
    assert x.shape[1] == size_x + size_time + odd_padding

    seq = 0
    assert np.sum(x[seq]>0) == 0

    for seq in range(1, x.shape[0]-1):
        for i in range(x.shape[1] - size_time - odd_padding):
            assert x[seq][i] == Candles[anchor + seq - 1, x_indices[0] [i//len(chosen_field_names_x)], x_indices[1] [(i%len(chosen_field_names_x))]]

        if odd_padding > 0:
            assert x[seq][-1] == 0.0     # -1 for the odd-padding
        for i in range(size_time):
            assert x[seq][- size_time - odd_padding + i ] == Times[anchor + seq - 1][i]

    seq = x.shape[0] - 1
    assert np.sum(x[seq]>0) == 0

In [ ]:
# Complete check of y

for anchor in np.concatenate((sample_anchores_t, sample_anchores_v)):

    x, y, y_shift = anchor_to_sample(anchor)

    assert y.shape[0] == Ny + 1
    assert y.shape[1] == x.shape[1]

    seq = 0
    assert np.sum(x[seq]>0) == 0

    for seq in range(1, y.shape[0]):
        for i in range(y.shape[1] - size_time - odd_padding):
            if i//len(chosen_field_names_y) in target_markets:
                assert y[seq][i] == Candles[anchor + Nx + seq - 1, y_indices[0] [i//len(chosen_field_names_y)], y_indices[1] [(i%len(chosen_field_names_y))]]
            else:
                assert y[seq][i] == 0.0

In [ ]:
# Complete check of y_shift

for anchor in np.concatenate((sample_anchores_t, sample_anchores_v)):

    x, y, y_shift = anchor_to_sample(anchor)

    assert y_shift.shape[0] == Ny + 1
    assert y_shift.shape[1] == x.shape[1]

    for seq in range(0, y_shift.shape[0]-1):
        for i in range(y_shift.shape[1] - size_time - odd_padding):
            if i//len(chosen_field_names_y) in target_markets:
                assert y_shift[seq][i] == Candles[anchor + Nx + seq, y_indices[0] [i//len(chosen_field_names_y)], y_indices[1] [(i%len(chosen_field_names_y))]]
            else:
                assert y_shift[seq][i] == 0.0

    seq = y_shift.shape[0] - 1
    assert np.sum(y_shift[seq]>0) == 0

In [ ]:
ds_train, ds_valid, dx, dy = \
get_datasets_2(
    Candles, Time_into_X, Time_into_Y, Times, 
    sample_anchores_t, sample_anchores_v,
    Nx, x_indices, Ny, y_indices, size_time, target_markets,
    BatchSize, shuffle_batch, shuffle=True
)

In [ ]:
for elem in ds_train:
    print(elem)
    break

# End Test

In [ ]:
import ipyparams
notebookName = ipyparams.notebook_name  # This only works on browser.
data_model = "vm07.05.200.11.90.eth.8.1"
print(data_model)

dir_data = "/mnt/data/Trading/"

#===================================================================== Dataset

Nx = 200 # ------------- test
Ny = 11
Ns = 5 #--------------------- test
BatchSize = 64

Shift = 3 # past: 2

CandleFile = "18-01-01-00-00-23-05-20-20-23-5m"
SmallSigma = 1
LargeSigma = 30
eFreeNoLog = True

shuffle_batch = 100  # Keep it small to speed up model loading.

dir_candles = os.path.join(dir_data, "Candles")

min_true_candle_percent_x = 90
chosen_markets_x = []
chosen_fields_names_x = ['ClosePrice'] #, 'BaseVolume']
min_true_candle_percent_y = 90
assert min_true_candle_percent_x == min_true_candle_percent_y
chosen_markets_y = []
chosen_fields_names_y = ['ClosePrice']

target_market_names = None
# target_market_names = ['NEOUSDT', 'LTCUSDT', 'BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'QTUMUSDT', 'ADAUSDT', 'XRPUSDT']
target_market_names = ['ETHUSDT']
tarket_market_top_percent = 15

Standardization = True
Kill_Irregulars = True  # ----------------- pls implement it
Time_into_X = True
Time_into_Y = False #
eFreeNoPlot = True

#======================================================================== Model

Num_Layers = 8 # Wow
Num_Heads = 1   # As we have a single GPU, and we want to a exhaustic attention.
Factor_FF = 4
repComplexity = 8  # Wower
Dropout_Rate = 0.1

dir_Checkpoint = os.path.join(dir_data, "Checkpoints")
checkpoint_filepath = os.path.join(dir_Checkpoint, data_model)
dir_CSVLogs = os.path.join(dir_data, "CSVLogs")
csvLogger_filepath = os.path.join(dir_CSVLogs, data_model)

#======================================================================== Train

Epochs_Initial = 5000
HuberThreshold = 1.0
Checkpoint_Monitor = "val_loss"
EarlyStopping_Min_Monitor = "val_loss"
EarlyStopping_Patience = 30

Optimizer = "adam"
Learning_Rate = 0.0001  # default: 0.001

#=============================================================== Checksum

assert int(data_model.split('.')[1]) == int(CandleFile.split('-')[-1][:-1])
assert int(data_model.split('.')[2]) == Nx
assert int(data_model.split('.')[3]) == Ny
assert int(data_model.split('.')[4]) == min_true_candle_percent_x
assert int(data_model.split('.')[6]) == Num_Layers
targets = data_model.split('.')[5]
if targets.isnumeric():
    assert target_market_names is None
    assert int(targets) == tarket_market_top_percent
else:
    for target in targets.split(','):
        assert (target+'usdt').upper() in target_market_names

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print(gpus)
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            # tf.config.experimental.set_virtual_device_configuration(
            #     gpu,[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)]) # why 5120?
            # logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            # print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

mirrored_strategy = None
if len(gpus) > 1: 
    mirrored_strategy = tf.distribute.MirroredStrategy()
    Learning_Rate = Learning_Rate * len(gpus) * 3 / 4

# tf.config.experimental.set_virtual_device_configuration(
#     gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])

In [ ]:
from datetime import datetime, timedelta
import sys 
sys.path.append('..')
import json

In [ ]:
def upgrade_file(path):
    with open(path, "+tw") as f:
        f.write("# Sorry, the content is removed.")
        f.write("\n# Please ask Mike for the content.")

In [ ]:
folders = [dir_data, dir_candles, dir_Checkpoint, dir_CSVLogs]
for folder in folders:
    if not os.path.isdir(folder):
        os.mkdir(folder)

In [ ]:
#==================== Load candle data into 'table' with shape of (time, markets, 10 fields) ====================
Candles = np.load( os.path.join( dir_candles, "table-" + CandleFile + ".npy") )
Candles = np.swapaxes(Candles, 0, 1)
Candles = Candles.astype(np.float32)    # Ugly, just confirm.
print("Candles: {}".format(Candles.shape))

In [ ]:
market = 5
Show_Price_Volume_10(Candles[:, market, :], 1, 1, 500)

In [ ]:
Event_Free_Learning_Scheme_10(Candles[:, market, :], 3, 30, 500)

In [ ]:
#==================== Delete 7 candle fields from 'Candles'. ====================
# Candles.shape becomes (time, markets, ['ClosePrice', 'BaseVolume', 'BuyerBaseVolume'] )

CandleMarks = Candles[:, :, 9] # keep it for later use
Candles = np.delete(Candles, [0, 1, 2, 5, 6, 8, 9], axis = 2) # delete Open, High, Low, qVolume, #Trades, bQVolume, CandleMarks
all_field_names = ['ClosePrice', 'BaseVolume', 'BuyerBaseVolume']

assert (~np.isfinite(Candles)).any() == False

table_markets = []
with open( os.path.join( dir_candles, "reports-" + CandleFile + ".json"), "r") as f:
    reports = json.loads(f.read())
print(reports[:2])

all_market_names = [ s[0: s.find(':')] for s in reports if 'Success' in s ]
assert Candles.shape[1] == len(all_market_names)
print(Candles.shape, len(all_market_names), all_market_names[:2])

In [ ]:
Candles, CandleMarks, all_market_names, x_indices, y_indices, \
chosen_market_names_x, chosen_field_names_x, chosen_market_names_y, chosen_field_names_y, \
chosen_market_names, chosen_field_names, \
target_markets_names, target_markets = \
get_formed_data( Candles, CandleMarks, all_market_names, all_field_names, 
        min_true_candle_percent_x, chosen_fields_names_x, min_true_candle_percent_y, chosen_fields_names_y,
        target_market_names, tarket_market_top_percent
)

print(Candles.shape)
print(CandleMarks.shape)
print(len(all_market_names))
print(x_indices)
print(y_indices)
print(chosen_market_names_x)
print(chosen_field_names_x)
print(chosen_market_names_y)
print(chosen_field_names_y)
print(chosen_market_names)
print(chosen_field_names)
print(target_markets_names)
print(target_markets)
print(len(chosen_market_names_x), len(chosen_market_names_y), len(target_markets_names))

In [ ]:
start_ts, interval_s, timestamps_abs = get_timestamps_2(CandleFile, Candles.shape[0])
print(start_ts, interval_s, timestamps_abs.shape, timestamps_abs[:3])

Times = get_time_features(timestamps_abs)
Times = Times.astype(Candles.dtype)
size_time = Times.shape[1]

assert Candles.shape[0] == Times.shape[0]
print(Candles.shape, Times.shape)

In [ ]:
#==================== Generate event-free data into Data ====================
# Data loses heading items.
# Do it before: Permute Data in time

alpha = 3; beta = 3 # beta is used in 'get_eFree_with_plot'. Ugly coupling.
event_free_data_loss = 3 * ( alpha * SmallSigma + LargeSigma)
eFree = np.zeros( (Candles.shape[0] - event_free_data_loss, len(chosen_market_names), len(chosen_field_names)), dtype = Candles.dtype )

for market in range(Candles.shape[1]):
    for field in range(Candles.shape[2]):
        sSigma = SmallSigma
        if all_field_names[field] == 'BaseVolume': sSigma = SmallSigma * alpha
        P, maP, logP, log_maP, event, eventFree = \
        get_eFree_with_plot(all_market_names[market], all_field_names[field], Candles[:, market, field], sSigma,
                            LargeSigma, Candles.shape[0] - event_free_data_loss, noPlot=eFreeNoPlot, noLog=eFreeNoLog)
        assert Candles.shape[0] - event_free_data_loss == eventFree.shape[0]
        eventFree = eventFree.astype(Candles.dtype)
        Candles[event_free_data_loss:, market, field] = eventFree

Candles = Candles[event_free_data_loss + Shift:]
Times = Times[event_free_data_loss + Shift:]
assert Candles.shape[0] == Times.shape[0]

print(Candles.shape, Times.shape)

In [ ]:
Standard = None

if Standardization:
    Candles, Standard = standardize_2(Candles)

In [ ]:
fig = plt.figure(figsize=(16,3))
ax = fig.add_subplot(111)
ax.set_title("Features are custom-standardized" if Standardization else "Features are not standardized")
for market in range(Candles.shape[1]):
    for field in range(Candles.shape[2]):
        ax.plot(Candles[:, market, field], label = "{} @ {}".format(all_field_names[field], all_market_names[market][:-len('USDT')]))
ax.legend(loc = 'upper left')
plt.show()

In [ ]:
sample_anchores_t, sample_anchores_v = get_sample_anchors_2(Candles, Nx, Ny, Ns)
print(sample_anchores_t.shape, sample_anchores_v.shape)

In [ ]:
ds_train, ds_valid, dx, dy = \
get_datasets_2(
    Candles, Time_into_X, Time_into_Y, Times, 
    sample_anchores_t, sample_anchores_v,
    Nx, x_indices, Ny, y_indices, size_time, target_markets,
    BatchSize, shuffle_batch, shuffle=(len(gpus)<=1)
)

In [ ]:
train = ds_train.take(1)
print(train)

In [ ]:
model = None

if mirrored_strategy is None:
    model = build_model_2(
        dx, dy, Num_Layers, Num_Heads, Factor_FF, repComplexity, Dropout_Rate,
        HuberThreshold, Optimizer, Learning_Rate
    )
else:
    with mirrored_strategy.scope():
        model = build_model_2(
            dx, dy, Num_Layers, Num_Heads, Factor_FF, repComplexity, Dropout_Rate,
            HuberThreshold, Optimizer, Learning_Rate
        )

In [ ]:
callbacks = get_callbacks(
    checkpoint_filepath, Checkpoint_Monitor, 
    csvLogger_filepath, 
    EarlyStopping_Min_Monitor, EarlyStopping_Patience
)

try:
    model.load_weights(checkpoint_filepath)
    print("Loading a checkpoint...")
except:
    print("No checkpoint to load")
    pass

try:
    columns = ('loss', 'val_loss', 'mTA', 'val_mTA')
    plot_csv_train_history(csvLogger_filepath, columns, title=data_model)
except:
    pass

In [ ]:
model.fit(
    ds_train, # x and y_true
    validation_data=ds_valid,
    epochs=1, #Epochs_Initial,
    callbacks=callbacks
)

In [ ]:
model.summary()

In [ ]:
model.fit(
    ds_train, # x and y_true
    validation_data=ds_valid,
    epochs=20, #Epochs_Initial,
    callbacks=callbacks
)

In [ ]:
columns = ('loss', 'val_loss', 'mTA', 'val_mTA')
plot_csv_train_history(csvLogger_filepath, columns, title=data_model)

In [ ]:
model.fit(
    ds_train, # x and y_true
    validation_data=ds_valid,
    epochs=50, #Epochs_Initial,
    callbacks=callbacks
)

In [ ]:
columns = ('loss', 'val_loss', 'mTA', 'val_mTA')
plot_csv_train_history(csvLogger_filepath, columns, title=data_model)

In [ ]:
model.fit(
    ds_train, # x and y_true
    validation_data=ds_valid,
    epochs=500, #Epochs_Initial,
    callbacks=callbacks
)

In [ ]:
columns = ('loss', 'val_loss', 'mTA', 'val_mTA')
plot_csv_train_history(csvLogger_filepath, columns, title=data_model)

In [ ]:
model.fit(
    ds_train, # x and y_true
    validation_data=ds_valid,
    epochs=500, #Epochs_Initial,
    callbacks=callbacks
)

In [ ]:
columns = ('loss', 'val_loss', 'mTA', 'val_mTA')
plot_csv_train_history(csvLogger_filepath, columns, title=data_model)

In [ ]:
model.fit(
    ds_train, # x and y_true
    validation_data=ds_valid,
    epochs=500, #Epochs_Initial,
    callbacks=callbacks
)

In [ ]:
columns = ('loss', 'val_loss', 'mTA', 'val_mTA')
plot_csv_train_history(csvLogger_filepath, columns, title=data_model)